*2021/01/07 Thu*

# 1-3. C++은 C 친구 - C와 공통점

cin, cout은 printf, scanf와 달리 &, %c 이런 것 구분할 필요 없음.
```
int user_input;
cin >> user_input;
```

# 2. C++ 참조자(레퍼런스)의 도입

* C Style Code

In [1]:
#include <iostream>

In [4]:
int change_val(int *p) {
    *p = 3;
    return 0;
}

In [ ]:
int main() {
    int number = 5;
    std::cout << number << std::endl;
    change_val(&number);
    std::cout << number << std::endl;
}
//main();

* C++ Code

In [9]:
int main() {
    int a = 3;
    int& another_a = a;
    
    another_a = 5; // 참조자 -> *()라고 생각!
    std::cout << "a : " << a << std::endl;
    std::cout << "another_a : " << another_a << std::endl;
    
    return 0;
}
main();

a : 5
another_a : 5


**레퍼런스는 반드시 처음에 누구의 별명이 될 것인지를 지정해야 한다.**

레퍼런스가 한 번 별명이 되면 절대로 다른 이의 별명이 될 수 없다. const
```
int &another_a; // 불가능
another_a = a; // a를 가리키는 게 아니라 그냥 값을 복사하는 것.
```

따라서 컴파일러상에서 a로 단순히 바꿔치기하면 되므로 포인터 변수처럼 따로 메모리에 영역을 둘 필요는 없음.

레퍼런스는 특별한 경우가 아닌 이상 메모리상에서 공간을 차지하지 않는다.

```
int x;
int& y = x; // x
int& z = y; // x
```

참조자의 참조자는 없음.

```std::cin >> user_input;```

user_input은 레퍼런스

```
int &ref = 4;
std::cout << ref << std::endl;
// error : cannot convert from 'int' to 'int &'
```

이게 가능하면 ```ref = 5;```와 같이 리터럴의 값을 바꾸는 행위가 가능.

따라서 상수 리터럴을 레퍼런스가 참조하는 것 역시 불가능.

대신 ```const int &ref = 4;```와 같이 상수 참조자로 선언하는 건 리터럴 참조 가능

레퍼런스의 레퍼런스, 레퍼런스의 배열, 레퍼런스의 포인터는 존재할 수 없다.

```
int &arr[2] = {a, b}; // illegal

int arr[3] = {1, 2, 3};
int (&ref)[3] = arr; // legal
ref[0] = 2, ref[1] = 3, ref[2] = 1;
```

지역 변수의 레퍼런스를 리턴하는 것은 컴파일 오류는 아니지만 경고. 그리고 아래 코드는 런타임 오류.

```
int &function() {
    int a = 2;
    return a;
}
int main() {
    int b = function();
    b = 3; // segmentation fault -> a가 사라졌으니까
    return 0;
}
```

아래와 같은 방식은 가능.

```
int& function(int &a) {
    a = 5;
    return a;
}
```

함수의 리턴값은 해당 문장이 끝난 후 바로 사라지는 값이므로 참조자를 만들면 바로 다음에 댕글링 레퍼런스가 되어 버린다.

(dangling pointer; 허상 포인터; 적절하고 유효한 객체를 가리키고 있지 않는 포인터)

-> 컴파일 오류.


```
int function() { int a = 5; return a; }
int main() {
    int &c = function();
    // error : cannot bind non-const lvalue reference of type 'int&' to an rvalue of type 'int'
    return 0;
}
```

그러나 ```const int &c = function();```였다면 가능.

상수 레퍼런스로 리턴 값을 받게 되면 해당 리턴 값의 생명이 연장됨. 연장되는 기간은 레퍼런스가 사라질 때까지.

![정리](./figures/Screen%20Shot%202021-01-07%20at%204.45.50%20PM.png)

## 생각 해보기

레퍼런스가 메모리 상에 반드시 존재해야 하는 경우는 어떤 경우가 있을까요? 그리고 메모리 상에 존재할 필요가 없는 경우는 또 어떤 경우가 있을 까요? (난이도 : 上)

```
int f() {}
const int &a = f();
```
와 같은 경우. -> 레퍼런스가 스코프 밖에서 사용된다면 메모리에 존재.

```int &b = a;```와 같은 경우.

# 3. C++의 세계로 오신 것을 환영합니다. (new, delete)

In [14]:
#include <iostream>

int main() {
    int *p = new int;
    *p = 10;
    
    std::cout << *p << std::endl;
    
    delete p;
    return 0;
}
main();

10


C++에서는 동적 할당에 대해 아예 키워드로써 제공. new는 sizeof도 필요 없고, 생성자를 자동으로 호출해서 초기화도 수행해줄 수 있음. (단, malloc의 경우는 realloc이란 함수로 재할당이 가능하지만, new는 불가하므로 객체가 아니라는 전제하에서는 할당 -> 복사 -> 해제의 과정이 필요. 객체가 아니라 재할당이 빈번하게 일어난다면 malloc/free가 더 좋은 선택이 될 수 있음.)

* 용법

```
T *ptr = new T;
delete ptr;
```

* heap 아닌 영역을 delete로 해제하려 하면

```
int a = 5;
delete &a; // 경고
```

In [6]:
/* new로 배열 할당하기 */

#include <iostream>

int main() {
    int arr_size;
    std::cout << "array size : ";
    std::cin >> arr_size;
    int *list = new int[arr_size];
    for (int i = 0; i < arr_size; i++) {
        std::cout << i << "th element of list : " << list[i] << std::endl;
    }
    delete[] list; // new []를 이용해서 할당했으니까 delete []
    return 0;
}
main();

array size : 

 3


0th element of list : 0
1th element of list : -536870912
2th element of list : -1372160997


* 컴파일러는 변수를 가장 가까운 scope부터 찾음.

In [5]:
int a = 4;
{
    std::cout << "외부의 변수 " << a << std::endl;
    int a = 3;
    std::cout << "내부의 변수 " << a << std::endl;
}

외부의 변수 4
내부의 변수 3
